# Zapytania HTTP i REST API - wprowadzenie

Jednym ze sposobów na komunikację z siecią/serwerami z poziomu skryptu napisanego w Pythonie
jest wykonywanie zapytań HTTP. Pozwala nam to zarówno na pobieranie danych z serwerów jak też na ich modyfikację
i dodawanie nowych informacji.

Wyróżniamy kilka podstawowych rodzajów zapytań HTTP:
- **GET** - pobranie danych
- **POST** - dodanie nowych danych
- **PUT**- aktualizacja istniejących danych
- **DELETE** - usunięcie danych

Współcześnie komunikacja z serwerami odbywa się za pośrednictwem tzw. **REST API**.
W dużym skrócie wygląda to w taki sposób, że wysyłamy zapytanie do serwera, a w odpowiedzi otrzymujemy
obiekt typu **JSON**.

Format **JSON** jest powszechnie wykorzystywany i w większości zastąpił format XML.
W formacie JSON obiekty przypominają obiekty w językach programowania.

Do wykonywania zapytań HTTP skorzystamy z biblioteki *requests*:

```pip install requests```

Najpierw importujemy bibliotekę

In [29]:
import requests

Teraz możemy wykonać proste zapytanie do serwera i zapisać odpowiedź w zmiennej *response*.

Dla przykładu skorzystamy ze strony http://api.open-notify.org która udostępnia API pozwalające pobrać
informacje o osobach przebywających obecnie w kosmosie, a także o położeniu ISS.

In [30]:
response = requests.get("http://api.open-notify.org/astros.json")

Odpowiedź z serwera składa się z kilku składowych. Jedną z nich jest *status code*.

In [31]:
print(response.status_code)

200


Kod ten informuje nas o statusie zwróconym przez nasze zapytanie. Kod **200** oznacza, że zapytanie
przebiegło pomyślnie.

Dla przykładu spróbujmy wykonać zapytanie pod nieistniejący adres.

In [32]:
response = requests.get("http://api.open-notify.org/non-existing")
print(response.status_code)

404


Otrzymaliśmy kod **404**. Oznacza to, że nie znaleziono na serwerze żądanego zasobu.

Wyróżniamy kilka grup kodów odpowiedzi:
- **1xx** - informacyjne
- **2xx** - sukces
- **3xx** - przekierowanie
- **4xx** - błąd klienta
- **5xx** - błąd serwera

Więcej informacji: https://pl.wikipedia.org/wiki/Kod_odpowiedzi_HTTP

Powtórzmy zapytanie i wypiszmy jego odpowiedź JSON.

In [33]:
response = requests.get("http://api.open-notify.org/astros.json")
print(response.json())

{'message': 'success', 'number': 7, 'people': [{'craft': 'ISS', 'name': 'Sergey Ryzhikov'}, {'craft': 'ISS', 'name': 'Kate Rubins'}, {'craft': 'ISS', 'name': 'Sergey Kud-Sverchkov'}, {'craft': 'ISS', 'name': 'Mike Hopkins'}, {'craft': 'ISS', 'name': 'Victor Glover'}, {'craft': 'ISS', 'name': 'Shannon Walker'}, {'craft': 'ISS', 'name': 'Soichi Noguchi'}]}


Dostaliśmy bogatą odpowiedź od serwera - listę osób przebywających w kosmosie.
Chcielibyśmy je wyświetlić w czytelniejszej formie. Do tego celu wykorzystamy wbudowaną bibliotekę **json**.

In [34]:
import json

def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

Teraz możemy użyć naszej funkcji do czytelnego wyświetlenia otrzymanej odpowiedzi.

In [35]:
jprint(response.json())

{
    "message": "success",
    "number": 7,
    "people": [
        {
            "craft": "ISS",
            "name": "Sergey Ryzhikov"
        },
        {
            "craft": "ISS",
            "name": "Kate Rubins"
        },
        {
            "craft": "ISS",
            "name": "Sergey Kud-Sverchkov"
        },
        {
            "craft": "ISS",
            "name": "Mike Hopkins"
        },
        {
            "craft": "ISS",
            "name": "Victor Glover"
        },
        {
            "craft": "ISS",
            "name": "Shannon Walker"
        },
        {
            "craft": "ISS",
            "name": "Soichi Noguchi"
        }
    ]
}


Jak można zauważyć, odpowiedź jest w formie słownika. Możemy łatwo przeiterować po wszystkich osobach
przebywających w kosmosie.

In [36]:
people = response.json()["people"]
for person in people:
    print(person["name"])

Sergey Ryzhikov
Kate Rubins
Sergey Kud-Sverchkov
Mike Hopkins
Victor Glover
Shannon Walker
Soichi Noguchi


Skorzystajmy teraz z drugiej funkcjonalności tego API - pozycji ISS.
Usługa pozwala nam dowiedzieć się, kiedy ISS będzie przelatywać nad zadaną pozycją.
W tym celu musimy przekazać odpowiednie parametry do zapytania - długość i szerokość geograficzną.
Tworzymy je w formacie zgodnym z formatem JSON.

In [37]:
parameters = {
    "lat": 53.013693144046606,
    "lon": 18.59777225190062
}

Przekazujemy parametry do zapytania.

In [38]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)
jprint(response.json())

{
    "message": "success",
    "request": {
        "altitude": 100,
        "datetime": 1607156326,
        "latitude": 53.013693144046606,
        "longitude": 18.59777225190062,
        "passes": 5
    },
    "response": [
        {
            "duration": 340,
            "risetime": 1607162805
        },
        {
            "duration": 600,
            "risetime": 1607168420
        },
        {
            "duration": 650,
            "risetime": 1607174177
        },
        {
            "duration": 653,
            "risetime": 1607179976
        },
        {
            "duration": 629,
            "risetime": 1607185779
        }
    ]
}


Czasy, jakie otrzymaliśmy w odpowiedzi zapisane są jako tzw. *timestamp* ablo *epoch*.
Możemy je łatwo przetworzyć na czytelny dla nas format korzystając z biblioteki datetime.

In [40]:
from datetime import datetime
for data in response.json()["response"]:
    time = datetime.fromtimestamp(data["risetime"])
    print(time)

2020-12-05 11:06:45
2020-12-05 12:40:20
2020-12-05 14:16:17
2020-12-05 15:52:56
2020-12-05 17:29:39
